In [1]:
import pandas as pd
import numpy as np
import datetime
import logging

from csh_fantasy_bot import bot
from csh_fantasy_bot.roster_change_optimizer import RosterChangeSet
from csh_fantasy_bot.nhl import score_team



In [2]:
# true value ignores actuals if they exist and works off projections only
simulation_mode=False
week_number = 5
league_id = '403.l.41177'
# league_id = "403.l.18782"
manager = bot.ManagerBot(week_number, league_id=league_id, simulation_mode=simulation_mode)
league_scorer = manager.score_comparer
manager.opp_team_name

"Kevan's Best Team"

In [3]:
# paste roster change text into here
"""
Here is example of what roster change text should look like
It is output from the rosterchangeset.pretty_print method

Date: 2021-02-08, in: Alexis Lafreniere(8641), out: Teuvo Teravainen(5698)
Date: 2021-02-09, in: Zdeno Chara(1700), out: Kailer Yamamoto(7537)
Date: 2021-02-10, in: Adam Fox(7174), out: Evander Kane(4684)
"""
roster_change_text="""
Date: 2021-02-18, in: Andrew Copp(6083), out: Oliver Ekman-Larsson(4686)
Date: 2021-02-15, in: Brock Nelson(4990), out: Nazem Kadri(4687)
Date: 2021-02-16, in: Adam Fox(7174), out: Tomas Hertl(5697)
"""

roster_changes = RosterChangeSet.from_pretty_print_text(roster_change_text, manager.all_player_predictions)
scores_with_changes = manager.score_team(roster_change_set=roster_changes)[1]
scoring_result = league_scorer.score(scores_with_changes)[manager.stat_categories]
print(scoring_result)
print(f"\nScore: opp->({scoring_result.loc['score_opp'].sum()}), league->({scoring_result.loc['score_league'].sum()})") 
if (roster_change_text != ''):
    scores_ignore_roster_changes = manager.score_team()[1]
    score_result_no_changes = league_scorer.score(scores_ignore_roster_changes)
    print (f"\nWithout roster changes: opp->({score_result_no_changes.loc['score_opp'].sum()}), league->({score_result_no_changes.loc['score_league'].sum()})") 

G       A    +/-     PIM      SOG       FW     HIT
my-scores          16.500  23.890  4.170  39.540  152.800  152.630  95.170
opponent           13.260  25.240  4.130  25.640  134.430  116.460  51.540
difference_opp      3.240  -1.350  0.040  13.900   18.370   36.170  43.630
mean-league        12.451  21.179  3.013  23.033  111.288  115.086  50.411
std dev             2.023   3.093  1.325   6.160   20.231   38.367  17.251
mean-opp           14.880  24.565  4.150  32.590  143.615  134.545  73.355
num_stds_opp        1.602  -0.436  0.030   2.257    0.908    0.943   2.529
score_opp           1.500  -0.436  0.030   1.500    0.908    0.943   1.500
win_loss_opp        1.000  -1.000  0.000   1.000    1.000    1.000   1.000
difference_league   4.049   2.711  1.157  16.507   41.512   37.544  44.759
score_league        1.500   0.877  0.873   1.500    1.500    0.979   1.500

Score: opp->(5.945), league->(8.729)

Without roster changes: opp->(1.165), league->(4.695)


In [5]:
scores_ignore_roster_changes = scores_with_changes
games_played = scores_ignore_roster_changes['G'].groupby(level=1).count()
result = manager.all_player_predictions.copy()
result['GP'] = games_played
gp_table = result[result.GP.notna()].sort_values('GP', ascending=False).round(2)
gp_table[['name', 'GP', 'fpts']]

,name,GP,fpts
player_id,,,
5985,Sean Monahan,4.0,0.38
5431,Vincent Trocheck,4.0,0.62
3637,Alex Ovechkin,4.0,1.27
5696,Tom Wilson,4.0,1.72
4990,Brock Nelson,4.0,0.88
5986,Darnell Nurse,4.0,0.43
5370,Dougie Hamilton,4.0,0.62
7174,Adam Fox,4.0,0.38
3358,Brent Burns,3.0,0.22


In [6]:
gp_table['GP'].sum()


51.0